### 라이브러리

In [4]:
import pandas as pd
import numpy as np

import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from wordcloud import WordCloud

from konlpy.tag import Okt
from konlpy import tag
from konlpy.corpus import kolaw
from konlpy.tag import Kkma, Komoran, Okt, Hannanum #Mecab은 윈도우에서 작동 불가능

from nltk.tokenize import sent_tokenize
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

### Lyrics 불러오기

In [9]:
Lyrics = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/odatt/Lyrics_v2.csv')

### 노래가사 이진분류(긍정, 부정)

In [88]:
# 행복, 희망 => 1(긍정), 슬픔, 분노 => 0(부정) 으로 이진분류

def rating_to_label(label):
    if label == 0 or label == 3:
        return 1
    else:
        return 0
    
Lyrics['y'] = Lyrics['label'].apply(lambda x: rating_to_label(x))
Lyrics_pos = Lyrics[Lyrics['y'] == 1]
Lyrics_neg = Lyrics[Lyrics['y'] == 0]


### 긍정, 부정 감성 함수 구현

In [94]:
def LyricsWordCount(arrList, emotion):

    pos_word = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/odatt/pos_pol_word.txt', index_col = 0)
    pos_word = pd.DataFrame(pos_word).reset_index()
    pos_word.columns = ['pos_word']

    neg_word = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/odatt/neg_pol_word.txt', index_col = 0)
    neg_word = pd.DataFrame(neg_word).reset_index()
    neg_word.columns = ['neg_word']

    list = []
    i=1

    # 감성 점수 구하기
    for Lyrics in arrList:
        neg_count = 0       # 부정 점수
        pos_count = 0       # 긍정 점수
        num = 0             # 감성 점수
        
        # 긍정 점수 구하기
        for pos_text in pos_word['pos_word']:
            pos_find = Lyrics.find(pos_text)
            if(pos_find > -1):
                pos_count += 1
            else: pos_count += 0


        # 부정 점수 구하기
        for neg_text in neg_word['neg_word']:
            neg_find = Lyrics.find(neg_text)
            if(neg_find > -1):
                neg_count += 1
            else: pos_count += 0

        # 감성 점수 공식
        if emotion == '긍정':
            num = pos_count - neg_count
        elif emotion == '부정':
            num = neg_count - pos_count

        srt = f"{i}번 노래 결과 -> 긍정 : {pos_count}, 부정 : {neg_count} , 감성 점수 : {num}"
        i = i + 1

        list.append(srt)

    return list

pos_result = LyricsWordCount(Lyrics_pos['Lyrics'], "긍정")
neg_result = LyricsWordCount(Lyrics_neg['Lyrics'], "부정")

print('긍정 노래')
for pos in pos_result:
    print(pos)

print('부정 노래')
for neg in neg_result:
    print(neg)

긍정 노래
1번 노래 결과 -> 긍정 : 11, 부정 : 6 , 감성 점수 : 5
2번 노래 결과 -> 긍정 : 6, 부정 : 0 , 감성 점수 : 6
3번 노래 결과 -> 긍정 : 3, 부정 : 3 , 감성 점수 : 0
4번 노래 결과 -> 긍정 : 6, 부정 : 3 , 감성 점수 : 3
5번 노래 결과 -> 긍정 : 8, 부정 : 1 , 감성 점수 : 7
6번 노래 결과 -> 긍정 : 13, 부정 : 3 , 감성 점수 : 10
7번 노래 결과 -> 긍정 : 4, 부정 : 4 , 감성 점수 : 0
8번 노래 결과 -> 긍정 : 11, 부정 : 2 , 감성 점수 : 9
9번 노래 결과 -> 긍정 : 23, 부정 : 11 , 감성 점수 : 12
10번 노래 결과 -> 긍정 : 6, 부정 : 3 , 감성 점수 : 3
11번 노래 결과 -> 긍정 : 10, 부정 : 5 , 감성 점수 : 5
12번 노래 결과 -> 긍정 : 0, 부정 : 0 , 감성 점수 : 0
13번 노래 결과 -> 긍정 : 6, 부정 : 2 , 감성 점수 : 4
14번 노래 결과 -> 긍정 : 9, 부정 : 2 , 감성 점수 : 7
15번 노래 결과 -> 긍정 : 9, 부정 : 5 , 감성 점수 : 4
16번 노래 결과 -> 긍정 : 6, 부정 : 4 , 감성 점수 : 2
17번 노래 결과 -> 긍정 : 3, 부정 : 0 , 감성 점수 : 3
18번 노래 결과 -> 긍정 : 5, 부정 : 4 , 감성 점수 : 1
19번 노래 결과 -> 긍정 : 3, 부정 : 1 , 감성 점수 : 2
20번 노래 결과 -> 긍정 : 3, 부정 : 10 , 감성 점수 : -7
21번 노래 결과 -> 긍정 : 8, 부정 : 7 , 감성 점수 : 1
22번 노래 결과 -> 긍정 : 7, 부정 : 4 , 감성 점수 : 3
23번 노래 결과 -> 긍정 : 9, 부정 : 7 , 감성 점수 : 2
24번 노래 결과 -> 긍정 : 5, 부정 : 1 , 감성 점수 : 4
25번 노래 결과 -> 긍정 : 14, 부정 : 3 , 감성